In [1]:
pip install catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 MB 6.7 MB/s eta 0:00:00


In [3]:
from catboost import CatBoostRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import numpy as np
import pandas as pd
# Libraries to help with reading and manipulating data
import pandas as pd
import numpy as np

from datetime import datetime

# Libaries to help with data visualization
import matplotlib.pyplot as plt
import seaborn as sns

# To tune model, get different metric scores, and split data
from sklearn.metrics import (
    f1_score,
    accuracy_score,
    recall_score,
    precision_score,
    confusion_matrix,
    roc_auc_score,
)
from sklearn.metrics import ConfusionMatrixDisplay

from sklearn import metrics

from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score

# To be used for data scaling and one hot encoding
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder

# To impute missing values
from sklearn.impute import SimpleImputer

# To oversample and undersample data
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler

# To do hyperparameter tuning
from sklearn.model_selection import RandomizedSearchCV

# To be used for creating pipelines and personalizing them
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

# To define maximum number of columns to be displayed in a dataframe
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

# To supress scientific notations for a dataframe
pd.set_option("display.float_format", lambda x: "%.3f" % x)

# To help with model building
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import (
    AdaBoostClassifier,
    GradientBoostingClassifier,
    RandomForestClassifier,
    BaggingClassifier,
)
from xgboost import XGBClassifier

# To suppress scientific notations
pd.set_option("display.float_format", lambda x: "%.3f" % x)

# To suppress warnings
import warnings
warnings.filterwarnings("ignore")

train_data = pd.read_csv('train_waves.csv') ##  read the data

test_data = pd.read_csv('test_waves.csv') ##  read the data


In [4]:
df=train_data.copy()


In [5]:
# Calculate the sine and cosine of wind direction
df['WDIR_sin'] = np.sin(np.radians(df['WDIR(degT)']))
df['WDIR_cos'] = np.cos(np.radians(df['WDIR(degT)']))
#wind speed cubed
df['WSCB']=np.power(df['WSPD(m/s)'],3)
#dominant wind power squared
df['DPD2']=np.power(df['DPD(sec)'],2)
#temperature differene between air and water
df['TDIF']=df['ATMP(degC)']-df['WTMP(degC)']
test_data['WDIR_sin'] = np.sin(np.radians(test_data['WDIR(degT)']))
test_data['WDIR_cos'] = np.cos(np.radians(test_data['WDIR(degT)']))
test_data['WSCB']=np.power(test_data['WSPD(m/s)'],3)
test_data['DPD2']=np.power(test_data['DPD(sec)'],2)
test_data['TDIF']=test_data['ATMP(degC)']-test_data['WTMP(degC)']


In [8]:
df.head()
df.shape

(3886, 22)

In [9]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
y=df['WVHT(m)']
df=df.drop(columns=['WVHT(m)'])
X = df.iloc[:, [2, 4, *range(6, df.shape[1])]]
X=sc.fit_transform(X)

In [10]:
from catboost import CatBoostRegressor
from sklearn.model_selection import GridSearchCV
model=CatBoostRegressor()
param_grid = {'iterations': [500, 1000, 1500],
              'learning_rate': [0.01, 0.05, 0.1],
              'depth': [4, 6, 8]}
# Instantiate the CatBoostRegressor
model = CatBoostRegressor()

# Instantiate GridSearchCV with the CatBoostRegressor and the parameter grid
grid_search = GridSearchCV(model, param_grid=param_grid, cv=5)

# Fit GridSearchCV to the training data
grid_search.fit(X, y)

# Print the best hyperparameters and the corresponding score
print('Best Hyperparameters: ', grid_search.best_params_)
print('Best Score: ', grid_search.best_score_)


Streaming output truncated to the last 5000 lines.
1002:	learn: 0.0141306	total: 12.2s	remaining: 6.03s
1003:	learn: 0.0141176	total: 12.2s	remaining: 6.01s
1004:	learn: 0.0140959	total: 12.2s	remaining: 6s
1005:	learn: 0.0140795	total: 12.2s	remaining: 5.99s
1006:	learn: 0.0140736	total: 12.2s	remaining: 5.98s
1007:	learn: 0.0140676	total: 12.2s	remaining: 5.96s
1008:	learn: 0.0140543	total: 12.2s	remaining: 5.95s
1009:	learn: 0.0140441	total: 12.2s	remaining: 5.94s
1010:	learn: 0.0140091	total: 12.3s	remaining: 5.93s
1011:	learn: 0.0139879	total: 12.3s	remaining: 5.92s
1012:	learn: 0.0139670	total: 12.3s	remaining: 5.9s
1013:	learn: 0.0139560	total: 12.3s	remaining: 5.89s
1014:	learn: 0.0139276	total: 12.3s	remaining: 5.88s
1015:	learn: 0.0139168	total: 12.3s	remaining: 5.86s
1016:	learn: 0.0138967	total: 12.3s	remaining: 5.85s
1017:	learn: 0.0138700	total: 12.3s	remaining: 5.84s
1018:	learn: 0.0138447	total: 12.3s	remaining: 5.83s
1019:	learn: 0.0138292	total: 12.4s	remaining: 5.81s

In [11]:
X_test=test_data.iloc[:, [2, 4, *range(6, df.shape[1])]]
X_test=sc.transform(X_test)
#X_test.head()

In [12]:
pred=grid_search.predict(X_test)


In [13]:
pred

array([0.32367969, 0.2644925 , 0.29732672, ..., 0.66946608, 1.28210931,
       1.29518477])

In [21]:
output=pd.DataFrame({'Id':test_data.ID,'WVHT(m)':pred})


In [23]:
output.head()

,Id,WVHT(m)
0,2019722240,0.324
1,2019722340,0.264
2,201973140,0.297
3,2019761040,0.258
4,2019761140,0.272


In [24]:
output.tail()

,Id,WVHT(m)
1530,2019102640,0.299
1531,2019102740,0.393
1532,2019102840,0.669
1533,2019102940,1.282
1534,20191021040,1.295


In [26]:
output.to_csv('submission_ML_ocean.csv',index=False)
print("Your submission was successful")

Your submission was successful
